In [ ]:
!pip install -q trino

In [1]:
# This CATALOG_URL works for the "docker compose" testing and development environment
# Change 'server' if you are not running on "docker compose" (f. ex. 'localhost' if Lakekeeper is running locally).
CATALOG_URL = "http://server:8181/catalog"
TRINO_URI = "http://trino:8080"
WAREHOUSE = "demo"

# Create Trino Catalog

In [2]:
from trino.dbapi import connect

conn = connect(host=TRINO_URI, user="trino")

In [4]:
cur = conn.cursor()
cur.execute(
    f"""
    CREATE CATALOG lakekeeper2 USING iceberg
    WITH (
        "iceberg.unique-table-location" = 'true',
        "iceberg.catalog.type" = 'rest',
        "iceberg.rest-catalog.uri" = '{CATALOG_URL}',
        "iceberg.rest-catalog.warehouse" = '{WAREHOUSE}',
        "iceberg.rest-catalog.security" = 'NONE',
        "iceberg.rest-catalog.vended-credentials-enabled" = 'true',
        "s3.region"= 'dummy',
        "s3.path-style-access" = 'true',
        "s3.endpoint" = 'http://minio:9000',
        "fs.native-s3.enabled" = 'true'
    )
"""
)

## Read and Write Tables

In [5]:
# Connect directly to demo catalog, so that we don't have to use it as a prefix
conn = connect(host=TRINO_URI, user="trino", catalog="lakekeeper2")
cur = conn.cursor()

In [33]:
cur.execute("CREATE SCHEMA IF NOT EXISTS trino_namespace")

In [ ]:
cur.execute(
    "CREATE TABLE trino_namespace.my_table (my_ints INT, my_floats DOUBLE, strings VARCHAR) WITH (format='PARQUET')"
)
cur.execute(
    "INSERT INTO trino_namespace.my_table VALUES (1, 1.0, 'a'), (2, 2.0, 'b')"
)

In [ ]:
cur.execute("SELECT * FROM trino_namespace.my_table").fetchall()

In [34]:
cur.execute("create table trino_namespace.test(my_ints INT, my_floats DOUBLE, strings VARCHAR) WITH (format='PARQUET')")

In [35]:
cur.execute("insert into trino_namespace.test VALUES (1, 1.0, 'a'), (2, 2.0, 'b')")

In [37]:
cur.execute("select * from trino_namespace.test").fetchall()

[[1, 1.0, 'a'], [2, 2.0, 'b']]

In [39]:
cur.execute("create view trino_namespace.test_view as select strings, count(1) cnt from lakekeeper.trino_namespace.test group by strings")

In [40]:
cur.execute("select * from trino_namespace.test_view").fetchall()

[['a', 1], ['b', 1]]

In [9]:
cur.execute("create or replace view trino_namespace.test_view as select strings, sum(my_floats) cnt from lakekeeper2.trino_namespace.test group by strings order by 2 desc")

In [8]:
cur.execute("select * from trino_namespace.test_view").fetchall()

[['b', 2.0], ['a', 1.0]]